# Load and preprocess data 

In [3]:
stop_words = set(["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"])

In [27]:
from pathlib import Path
import re

def rm_stop(words):  # remove stop words from list of words
    return [w for w_ in w.split() if w_ not in stop_words]

def preprocess(w):  # remove non-ASCII and lower characters from w
    return rm_stop(re.sub("[^a-zA-Z ]", '', w).lower())

class InstaCities:
    def __iter__(self):
        for f in Path("../InstaCities1M").glob("**/*.txt"):
            yield preprocess(f.read_text())

# Word2Vec model

In [28]:
import gensim.models

sentences = InstaCities()
model = gensim.models.Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=5, workers=4)
model.save("model_insta_cities")

In [39]:
wv = model.wv
wv.most_similar('city', topn=10)

[('town', 0.637067973613739),
 ('buildings', 0.6352139115333557),
 ('streets', 0.6173369288444519),
 ('place', 0.6166399717330933),
 ('cities', 0.6086874008178711),
 ('building', 0.5980123281478882),
 ('world', 0.5960747003555298),
 ('citys', 0.5865464806556702),
 ('skyline', 0.581667423248291),
 ('downtown', 0.5648966431617737)]

# Comparison with Google News model

In [40]:
import gensim.downloader as api
wv_news = api.load('word2vec-google-news-300')

In [78]:
from pprint import pprint

pprint(wv_news.most_similar(positive=["car", "sports", "hairstyle", "food", "toronto"], topn=5))
pprint(model.wv.most_similar(positive=["car", "sports", "hairstyle", "food", "toronto"], topn=5))

[('DeVillers_reports', 0.5512474179267883),
 ('limited_edition_MGTF', 0.5153472423553467),
 ('e_mail_mikenadel@sbcglobal.net', 0.5088345408439636),
 ('TAKE_TWO_An', 0.4983653426170349),
 ('KTEN_Deeda_Payton', 0.48801666498184204)]
[('cool', 0.5977016091346741),
 ('sport', 0.5970734357833862),
 ('cars', 0.5735954642295837),
 ('fitness', 0.5330854058265686),
 ('fun', 0.5290842056274414)]


In [80]:
import numpy as np

top = 7

def similar(wv, *words):
    v = np.sum(np.array(wv.get_vector(w) for w in words))
    return [w[0] for w in wv.similar_by_vector(v, topn=top)]

for w1, w2 in [("toronto", "wild"), ("happy", "family"), ("food", "healthy"), ("food", "sweet")]:
    print(f"Most similar words to {w1} + {w2}:")
    print(f"InstaCities: {similar(wv, w1, w2)}")
    print(f"Google News: {similar(wv_news, w1, w2)}\n")

print(f"Most similar words to food - healthy:")
print(f"InstaCities: {[w[0] for w in wv.similar_by_vector(wv.get_vector('food') - wv.get_vector('healthy'), topn=top)]}")
print(f"Google News: {[w[0] for w in wv_news.similar_by_vector(wv_news.get_vector('food') - wv_news.get_vector('healthy'), topn=top)]}")

Most similar words to toronto + wild:
InstaCities: ['shiamak', 'prenup', 'mattyctattoo', 'estonian', 'goldnova', 'supportplease', 'illtattoogmailcom']
Google News: ['Seashore_Trolley_Museum', 'Inc._OTCBB_AMOR', 'cyclic_steaming', 'CaFCP', 'ARBOC_Mobility', 'CNHA', 'SunLine_Transit']

Most similar words to happy + family:
InstaCities: ['todaydailykoreaselfieinstadailytravelinstatraveltravelgrammelbourneaustraliaweekend', 'tigres', 'grindseason', 'algorithms', 'perthlife', 'coworkingspaces', 'powai']
Google News: ['SB####_ONX', 'boringly', 'Initiates_Independent_Research', 'PDGFR', 'giddily', 'Cbl', '##c###']

Most similar words to food + healthy:
InstaCities: ['fiestainfantil', 'monumentvalley', 'salinas', 'fremont', 'cuu', 'ghardaia', 'stockton']
Google News: ['Rollingstone.com', 'Sveriges_Radio_SR', 'Lif', 'Onion_AV_Club', 'Merryday', 'MEGADETH_drummer', 'Laiho']

Most similar words to food + sweet:
InstaCities: ['diamondwholesaler', 'tudungcomel', 'nycla', 'dubaibasedmakeupartist', '

In [74]:
print(wv.doesnt_match(["man", "woman", "kid", "dog"]))
print(wv_news.doesnt_match(["man", "woman", "kid", "dog"]))

dog
dog


In [75]:
print(wv.similarity('woman', 'man'))
print(wv_news.similarity('woman', 'man'))

0.70655835
0.76640123
